# CLASSIFICATORE SVM LINEARE CON INPUT DI WORD EMBEDDINGS

Classificatore basato su svm lineare che prende in input i word embeddings di parole italiane, più specificamente dei word embedding sviluppati per EvalIta 2018 dall'Italian NLP Lab. 

***

In [19]:
import numpy as np 
import re 
import os

### Caricamento Word Embeddings

Il file txt dei word embedding è stato ottenuto attraverso l'elaborazione di un file sqlite. Per ogni riga (la quale rappresenta una parola) all'interno del file, si crea un record in un dizionario con chiave parola e come valore il vattore che rappresenta il word embedding.

In [20]:
#funzione per il caricamento degli embedding itwac
def load_embeddings():
    embeddings_dict = dict()
    for line in open('../data/embeddings/itwac32.txt', encoding='utf-8'):
        splitted = line.strip().split('\t')
        word = splitted[0]
        embedding = splitted[1:]
        embedding = [float(val) for val in embedding]
        embeddings_dict[word] = np.asarray(embedding)
    return embeddings_dict

embeddings = load_embeddings()


In [21]:
embeddings['amico']

array([-0.02069134,  0.09119736,  0.25785723, -0.23561105, -0.28197852,
       -0.13193955, -0.13197723, -0.05229089, -0.28881341,  0.06564969,
       -0.30802506,  0.11779311, -0.03571652, -0.08748714, -0.24729131,
        0.2577146 ,  0.11925782, -0.27795964,  0.20498367,  0.08414506,
        0.08175091, -0.05181665, -0.34403449, -0.05261306,  0.08858071,
       -0.09748928,  0.12879393,  0.04387102, -0.04690454,  0.08181785,
        0.321078  ,  0.01658573])

### Normalizzazione del testo 

Le parole all'interno dei post devono essere normalizzate in questo modo:

Numeri:
- interi tra 0 e 2100 possono essere mantenuti così come sono
- i numeri interi maggiori di 2100 diventano una stringa specifica con un numero che rappresenta il numero di cifre
- se non si tratta di numeri interi, si convertono le cifre all'interno della stringa in questa sequenze @Dg

Parole:
- le parole che iniziano con una lettera maiuscola devono avere prima parola maiuscola e le altre minuscole
- le parole che iniziano con una lettera minuscola devono avere tutti i caratteri minuscolo

In [22]:
#funzione che trasforma le cifre all'interno di un token
def digit_norm(word):
    try:
        val = int(word)
    except:
        normalized = re.sub('\d', '@Dg', word)
        return normalized
    if val >= 0 and val<1200:
        return str(val)
    else:
        return f'DIGLEN_{str(len(str(val)))}'

#funzione che normalizza i token
def normalize(word):
    if "http" in word or ("." in word and "/" in word):
        return str("___URL___")
    if len(word)>26:
        return "__LONG-LONG__"
    digits = digit_norm(word)
    if digits != word:
        word = digits
    if word[0].isupper():
        word = word.capitalize()
    else:
        word = word.lower()
    return word



In [23]:
#funzione che estrae dai file conllu i token e li normalizza
def get_tokens(doc_path):
    doc_tokens = []
    skip_lines = 0
    first = False
    for line in open(doc_path, "r", encoding="utf-8"):
        splitted = line.strip().split("\t")
        if line[0].isdigit():
            if skip_lines == 0 and first == False:
                if "-" not in splitted[0]:
                    word = normalize(splitted[1])
                    pos=splitted[3]
                    token = {'word': word, 'pos': pos}
                    doc_tokens.append(token)
                else:
                    word=normalize(splitted[1])
                    skip_lines = len(splitted[0].split('-'))
                    first = True
            elif skip_lines != 0 and first == True:
                pos = splitted[3]
                token = {'word': word, 'pos': pos}
                doc_tokens.append(token)
                skip_lines-=1
                first = False
            else: 
                skip_lines-=1
    return doc_tokens

#funzione che crea il traingset o test set in base all'argomento specificato
def create_set(type):
    annotated_posts = []
    for doc in os.listdir("../data/UD_annotation"):
        if type in doc: 
            doc_path = "../data/UD_annotation/" + doc    
            doc_tokens = get_tokens(doc_path)
            annotated_posts.append(doc_tokens)
    return annotated_posts


print(get_tokens("../data/UD_annotation/training#125642756147265536#0#0#TW-SENTIPOLC.conllu"))

training_set = create_set("training")
test_set = create_set("test")

[{'word': 'Le', 'pos': 'DET'}, {'word': '5', 'pos': 'NUM'}, {'word': 'sgradevoli', 'pos': 'ADJ'}, {'word': 'realtà', 'pos': 'NOUN'}, {'word': 'di', 'pos': 'ADP'}, {'word': 'cui', 'pos': 'PRON'}, {'word': 'Berlusconi', 'pos': 'PROPN'}, {'word': 'dovrebbe', 'pos': 'AUX'}, {'word': 'rendersi', 'pos': 'VERB'}, {'word': 'personalmente', 'pos': 'ADV'}, {'word': 'conto', 'pos': 'NOUN'}, {'word': '<url>', 'pos': 'PROPN'}, {'word': 'Mario', 'pos': 'PROPN'}, {'word': 'Monti', 'pos': 'PROPN'}, {'word': 'non', 'pos': 'ADV'}, {'word': 'usa', 'pos': 'VERB'}, {'word': 'mezzi', 'pos': 'ADJ'}, {'word': 'termini', 'pos': 'NOUN'}]


In [24]:
training_set[2]

[{'word': '#la@dg', 'pos': 'NUM'},
 {'word': 'ma', 'pos': 'CCONJ'},
 {'word': 'perche', 'pos': 'ADV'},
 {'word': "'", 'pos': 'PUNCT'},
 {'word': 'Mario', 'pos': 'PROPN'},
 {'word': 'Monti', 'pos': 'PROPN'},
 {'word': 'non', 'pos': 'ADV'},
 {'word': 'fa', 'pos': 'VERB'},
 {'word': 'il', 'pos': 'DET'},
 {'word': 'premier', 'pos': 'NOUN'},
 {'word': '?', 'pos': 'PUNCT'},
 {'word': 'Che', 'pos': 'DET'},
 {'word': 'persona', 'pos': 'NOUN'},
 {'word': 'competente', 'pos': 'ADJ'},
 {'word': 'e', 'pos': 'CCONJ'},
 {'word': 'per', 'pos': 'ADP'},
 {'word': 'bene', 'pos': 'ADV'},
 {'word': '!', 'pos': 'PUNCT'}]

### Pre-processing

Per fare in modo di avere un vettore di features per ogni sample del nostro dataset, per ogni post vanno estratti gli embedding e aggregati secondo una delle seguenti strategie:
- Somma 
- Media 
- Prodotto

In [45]:
#funzione che restituisce gli embedding all'interno di un post e infine li aggrega secondo la strategia definita
def get_post_embeddings(post, aggr, consider_pos=[]):
    post_emb = []
    for token in post:
        word = token['word']
        pos = token['pos']
        if consider_pos == []: 
            if word in embeddings:
                single_embedding = embeddings[word]
                post_emb.append(single_embedding)
        else:
            if word in embeddings and pos in consider_pos:
                single_embedding = embeddings[word]
                post_emb.append(single_embedding)
    print(len(post_emb))
    if len(post_emb)==0:
        post_emb = [np.zeros(32)]
    if aggr=="sum":
        return post_embeddings_sum(post_emb)
    if aggr=="mean":
        return post_embeddings_mean(post_emb)
    if aggr=="prod":
        return post_embeddings_prod(post_emb)

#funzione che estrae gli embeddings e li aggrega separatamente per part of speech; infine i tre array vengono concatenati
def get_post_embeddings_sep(post, aggr, pos_to_consider):
    all_embs = []
    for pos in pos_to_consider:
        pos_embs = []
        for token in post:
            word = token['word'] 
            if token['pos'] == pos and word in embeddings:
                pos_embs.append(embeddings[word])
        if len(pos_embs) == 0:
            pos_embs = [np.zeros(32)]
        if aggr=="sum":   
            pos_aggr = post_embeddings_sum(pos_embs)
        if aggr=="mean":
            pos_aggr = post_embeddings_mean(pos_embs)
        if aggr=="prod":
            pos_aggr == post_embeddings_prod(pos_embs)
        all_embs.append(pos_aggr)
    all_pos_embs = np.concatenate(all_embs)
    return all_pos_embs

def post_embeddings_sum(post_emb):
    return np.sum(post_emb, axis=0)

def post_embeddings_mean(post_emb):
    embeddings_sum = post_embeddings_sum(post_emb)
    return np.divide(embeddings_sum, len(post_emb))

def post_embeddings_prod(post_emb):
    return np.prod(post_emb, axis=0)

sample = get_post_embeddings_sep(training_set[0], "sum", ['ADJ', 'NOUN', 'VERB'])
print(sample)

#print(f'Media: {post_embeddings_mean(sample)}')
#print(f'Somma: {post_embeddings_sum(sample)}')
#print(f'Prodotto: {post_embeddings_prod(sample)}')

[ 0.33562477 -0.05448548 -0.42705861  0.01437109  0.45159502  0.00348032
  0.26449129  0.03243306 -0.08381133 -0.14544163 -0.05085387 -0.11924932
  0.38293768 -0.0571368   0.44839092 -0.33565044  0.37418649  0.02131738
  0.25892133 -0.28732456  0.60670963  0.151251   -0.54009455  0.04013886
  0.04687114 -0.11289249 -0.48268102 -0.13149872 -0.32601021  0.02917017
 -0.17283661 -0.01268265 -0.03171508 -0.05594792 -0.13454684  0.43743487
  0.52703507 -0.03730955  0.55663195 -0.18791406  0.14612699 -0.01575376
 -0.11765517 -0.19878786  0.44469127  0.07014844  0.11291848 -0.49556313
 -0.13480377  0.41072911 -0.35959498 -0.41556786  0.53653162  0.25484272
 -0.73539679  0.2500334  -0.03019049 -0.71849942  0.12890437 -0.08805762
 -0.34093672 -0.46701554 -0.36786842 -0.03138301  0.43814081 -0.10103156
 -0.19417439  0.13527741 -0.19628561 -0.04956499  0.22108051 -0.14000706
 -0.08482647  0.04375442  0.08646346 -0.00164628  0.11506289  0.08473135
 -0.06659641  0.10320415  0.23755038 -0.05376894  0